In [14]:
# 행정동 별 실 거주 인구 데이터의 양이 많아 Huggingface에 업로드 해 두었습니다. 아래의 링크에 들어가 LocalPeople.zip 파일을 받아주셔야 합니다.
# 데이터를 다운 후 압축을 해제하시고 Data 폴더 안으로 옮겨주세요.
# https://huggingface.co/datasets/uhjin1130/LocalPeople/blob/main/LocalPeople.zip

In [15]:
import os
import pandas as pd
from sklearn.impute import SimpleImputer

# 실거주 인구 데이터 로딩
def load_people_data(year, base_path):
    files = sorted([
        f for f in os.listdir(base_path)
        if f.startswith(f'LOCAL_PEOPLE_DONG_{year}') and f.endswith('.csv')
    ])
    df_list = []
    for f in files:
        path = os.path.join(base_path, f)
        try:
            df = pd.read_csv(path, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(path, encoding='cp949')

        def looks_like_dongcode(series):
            return series.astype(str).str.match(r'^11\d{6}$').sum()

        score1 = looks_like_dongcode(df['행정동코드'])
        score2 = looks_like_dongcode(df['시간대구분'])

        if score2 > score1:
            df['행정동코드'] = df['시간대구분'].astype(int).astype(str).str.zfill(8)
        else:
            df['행정동코드'] = df['행정동코드'].astype(float).astype(int).astype(str).str.zfill(8)

        df_list.append(df)

    if not df_list:
        return pd.DataFrame()

    df_year = pd.concat(df_list)
    df_year = df_year[df_year['행정동코드'].notna()]
    df_grouped = df_year.groupby('행정동코드').mean(numeric_only=True).reset_index()
    return df_grouped

# 상권 분석 데이터 로딩
def load_biz_data(year, base_path):
    path = os.path.join(base_path, f'Trading_Area_{year}.csv')
    if not os.path.exists(path):
        return pd.DataFrame()
    df = pd.read_csv(path, encoding='utf-8')
    df['행정동_코드'] = df['행정동_코드'].astype(str).str.zfill(8)
    df_grouped = df.groupby(['행정동_코드', '서비스_업종_코드_명'])['당월_매출_금액'].mean().reset_index()
    return df_grouped

# 연도별 분석
def analyze_year(year, people_dir, biz_dir, save_dir):
    people_df = load_people_data(year, people_dir)
    biz_df = load_biz_data(year, biz_dir)
    if people_df.empty or biz_df.empty:
        return None

    merged = pd.merge(biz_df, people_df, left_on='행정동_코드', right_on='행정동코드', how='inner')
    if merged.empty:
        return None

    null_ratio = merged.isnull().mean()
    drop_cols = null_ratio[null_ratio > 0.5].index.tolist()
    merged = merged.drop(columns=drop_cols)

    X = merged.drop(columns=['행정동_코드', '서비스_업종_코드_명', '당월_매출_금액', '행정동코드'])
    y = merged['당월_매출_금액']
    info = merged[['행정동_코드', '서비스_업종_코드_명']].copy()

    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

    info['총매출'] = y.values
    info['연도'] = year
    info['순위'] = info.groupby('행정동_코드')['총매출'].rank(method='dense', ascending=False)

    if year > 2019:
        prev_path = os.path.join(save_dir, f'Analyze_{year - 1}.csv')
        if os.path.exists(prev_path):
            prev_df = pd.read_csv(prev_path, encoding='utf-8-sig')
            prev_df['행정동코드'] = prev_df['행정동코드'].astype(str).str.zfill(8)
            prev_df['업종'] = prev_df['업종'].astype(str)
            info = pd.merge(info, prev_df[['행정동코드', '업종', '총매출']],
                            left_on=['행정동_코드', '서비스_업종_코드_명'],
                            right_on=['행정동코드', '업종'], how='left', suffixes=('', '_prev'))
            info['매출증감률'] = ((info['총매출'] - info['총매출_prev']) / info['총매출_prev']) * 100
            info['매출증감률'] = info['매출증감률'].fillna(0)
        else:
            info['매출증감률'] = 0
    else:
        info['매출증감률'] = 0

    result = info[['연도', '행정동_코드', '서비스_업종_코드_명', '총매출', '매출증감률', '순위']].copy()
    result.rename(columns={'행정동_코드': '행정동코드', '서비스_업종_코드_명': '업종'}, inplace=True)
    result.to_csv(os.path.join(save_dir, f'Analyze_{year}_LocalPeople.csv'), index=False, encoding='utf-8-sig')
    return result

# 실행
def run_all(people_dir, biz_dir, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    all_data = []

    for year in range(2019, 2025):
        result = analyze_year(year, people_dir, biz_dir, save_dir)
        if result is not None:
            all_data.append(result)
        else:
            print(f"{year}년 분석 실패")

    if not all_data:
        raise ValueError("분석 데이터가 존재하지 않아 학습할 수 없습니다.")

# 실행
run_all('../../Data/LocalPeople', '../../Data/Trading_Area', './outputs')


In [16]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 로드
years = list(range(2019, 2025))
df_list = []
for y in years:
    df = pd.read_csv(f'./outputs/Analyze_{y}_LocalPeople.csv')
    df["연도"] = y
    df["행정동코드"] = df["행정동코드"].astype(str).str.zfill(8)
    df_list.append(df)
data = pd.concat(df_list, ignore_index=True)

# 2. 매핑 테이블 로드
dong_map = pd.read_excel('../../Data/행정동코드_매핑정보.xlsx', header=1)
dong_map = dong_map[['H_DNG_CD', 'H_DNG_NM']]
dong_map.columns = ['행정동코드', '행정동명']
dong_map['행정동코드'] = dong_map['행정동코드'].astype(str).str.zfill(8)

# 3. 범주형 변수 인코딩
le_dong = LabelEncoder()
le_biz = LabelEncoder()

data['행정동코드_enc'] = le_dong.fit_transform(data['행정동코드'])
data['업종_enc'] = le_biz.fit_transform(data['업종'])

X = data[['연도', '행정동코드_enc', '업종_enc']]
y = data['총매출']

# 4. 모델 학습
model = RandomForestRegressor(
    n_estimators=500,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
model.fit(X, y)

# 5. 2025년 예측용 조합 생성
dong_list = sorted(data['행정동코드'].unique())
biz_list = sorted(data['업종'].unique())
pred_df = pd.DataFrame([
    {"연도": 2025, "행정동코드": d, "업종": b}
    for d in dong_list for b in biz_list
])
pred_df['행정동코드_enc'] = le_dong.transform(pred_df['행정동코드'])
pred_df['업종_enc'] = le_biz.transform(pred_df['업종'])

# 6. 예측
X_pred = pred_df[['연도', '행정동코드_enc', '업종_enc']]
pred_df['예측_총매출'] = model.predict(X_pred)

# 7. 행정동명 병합 및 매출 증감률 반영
pred_df = pred_df.merge(dong_map, on='행정동코드', how='left')
avg_rate = data.groupby(['행정동코드', '업종'])['매출증감률'].mean().reset_index()
avg_rate.columns = ['행정동코드', '업종', '과거_평균_증감률']
pred_df = pred_df.merge(avg_rate, on=['행정동코드', '업종'], how='left')
pred_df['과거_평균_증감률'] = pred_df['과거_평균_증감률'].fillna(0)

# 8. 점수 계산 및 순위
pred_df['예측_점수'] = pred_df['예측_총매출'] * (1 + pred_df['과거_평균_증감률'])
pred_df['순위'] = pred_df.groupby('행정동코드')['예측_점수'].rank(ascending=False, method='min')
pred_df['연도'] = 2025

# 9. 최종 저장
final = pred_df[['연도', '행정동코드', '행정동명', '업종', '예측_총매출', '순위']]
final = final.sort_values(by=['행정동코드', '순위'])
final.to_csv('./Predicted_2025_Top_Business_LocalPeople_RF.csv', index=False, encoding='utf-8-sig')

final.head(10)


,연도,행정동코드,행정동명,업종,예측_총매출,순위
57,2025,11110515,청운효자동,한식음식점,6.181712e+09,1.0
52,2025,11110515,청운효자동,커피-음료,4.762092e+09,2.0
46,2025,11110515,청운효자동,조명용품,4.608279e+09,3.0
16,2025,11110515,청운효자동,서적,3.394097e+09,4.0
27,2025,11110515,청운효자동,양식음식점,3.045003e+09,5.0
20,2025,11110515,청운효자동,슈퍼마켓,1.964775e+09,6.0
34,2025,11110515,청운효자동,의료기기,1.692901e+09,7.0
55,2025,11110515,청운효자동,편의점,1.628359e+09,8.0
53,2025,11110515,청운효자동,컴퓨터및주변장치판매,1.448529e+09,9.0
32,2025,11110515,청운효자동,운동/경기용품,1.170039e+09,10.0
